In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import csv
import requests
import json
import re

In [ ]:
def get_blog_url(soup):
  div_ = soup.find_all('div',attrs={'class': 'FL PR20'})
  url_list = []
  for title in div_:
    href = title.find('a')['href']
    url_list.append("https://www.moneycontrol.com/"+href)
  return url_list

In [ ]:
def get_blog_content(url):
  request = requests.get(url)
  soup = BeautifulSoup(request.text,'html.parser')
  all_scripts = soup.find_all('script',attrs={'type': 'application/ld+json'})
  raw_article_str = all_scripts[2].get_text().replace('\r\n',' ')
  parts = re.split(r"""("[^"]*"|'[^']*')""",raw_article_str)
  parts[::2] = map(lambda s: "".join(s.split()),parts[::2])
  article_str = "".join(parts)
  article_str = article_str[1:]
  article_str = article_str[:-1]
  article_dict = json.loads(article_str)
  all_tags = soup.find_all('div', attrs = {'class':'tags_first_line'})
  lst_all_tags = []
  for i in all_tags:
    lst_all_tags.append(i.get_text())
  tags = lst_all_tags[0].replace('TAGS:','')
  tags = tags.replace('\n','')
  tags = tags.split('#')
  tags = tags[1:]
  tags = ', '.join([str(elem).strip() for elem in tags])
  article_dict['tags'] = tags
  return article_dict

In [ ]:
def get_page_no(url,sc_id,page_no,next,year):
  request = requests.get(url)
  soup = BeautifulSoup(request.text, 'html.parser')
  all_page_no = soup.find_all('div', attrs = {'class':'pages MR10 MT15'})
  page_list = [i.text for i in all_page_no[0].find_all('a')]
  if len(page_list)==0:
    return 1,next
  if(any(map(str.isdigit,page_list[-1]))):
    return int(page_list[-1]), next
  else:
    next = next + 1
    page_no = int(page_list[-2])
    url = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id="+sc_id+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
    return get_page_no(url,sc_id,page_no,next,year)

In [ ]:
def save_company_data(url_ = "https://www.moneycontrol.com/stocks/company_info/stock_news.php?",sc_id=[], page_no=1, next=0, years=[]):
  for company in sc_id:
    df = pd.DataFrame(columns = ['company', 'datePublished', 'author', 'headline', 'description', 'articleBody', 'tags', 'url'])
    cnt = 20
    status = False
    for year in years:
      if status==True:
        break
      print('year: ', year)
      print('page_no: ', page_no)
      print('next: ', next)
      url = url_ + "sc_id="+company+"&scat=&pageno="+str(page_no)+"&next="+str(next)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
      print('url: ', url)

      max_page_no, max_next = get_page_no(url, company, page_no, next, year)
      max_next = max_next + 1


      for i in range(max_next):
        for j in range((i*10)+1, (i*10)+11):
          if status == True:
            break
          if j <= max_page_no:
            url_list = []
            url = url_ + "sc_id="+company+"&scat=&pageno="+str(j)+"&next="+str(i)+"&durationType=Y&Year="+str(year)+"&duration=1&news_type="
            request = requests.get(url)
            soup = BeautifulSoup(request.text, 'html.parser')
            url_list = get_blog_url(soup)



            for url in url_list:
              try:
                if cnt > 0:
                  article_dict = get_blog_content(url)
                  print(company)
                  print(article_dict['datePublished'])
                  print(article_dict['author'])
                  print(article_dict['headline'])
                  print(article_dict['description'])
                  print(article_dict['articleBody'])
                  print(article_dict['tags'])
                  print(article_dict['url'])
                  print('-------------------------------------------------------------')

                  article_lst = [[company,
                                  article_dict['datePublished'],
                                  article_dict['author'],
                                  article_dict['headline'],
                                  article_dict['description'],
                                  article_dict['articleBody'],
                                  article_dict['tags'],
                                  url]]
                  df = pd.concat([df,(pd.DataFrame(article_lst, columns = ['company', 'datePublished', 'author', 'headline', 'description', 'articleBody', 'tags', 'url']))], ignore_index = True)
                  cnt = cnt - 1

                else:
                  status= True
                  break

              except:
                # article_lst = [[company,'error','error','error','error','error','error',url]]
                # df = pd.concat([df,(pd.DataFrame(article_lst, columns = ['company', 'datePublished', 'author', 'headline', 'description', 'articleBody', 'tags', 'url']))], ignore_index = True)

                continue
          else:
            break

    df.to_csv('/content/'+company+'.csv')

In [ ]:
save_company_data(sc_id=['PSL'],years=[2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005])

year:  2024
page_no:  1
next:  0
url:  https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=PSL&scat=&pageno=1&next=0&durationType=Y&Year=2024&duration=1&news_type=
year:  2023
page_no:  1
next:  0
url:  https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=PSL&scat=&pageno=1&next=0&durationType=Y&Year=2023&duration=1&news_type=
year:  2022
page_no:  1
next:  0
url:  https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=PSL&scat=&pageno=1&next=0&durationType=Y&Year=2022&duration=1&news_type=
year:  2021
page_no:  1
next:  0
url:  https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=PSL&scat=&pageno=1&next=0&durationType=Y&Year=2021&duration=1&news_type=
year:  2020
page_no:  1
next:  0
url:  https://www.moneycontrol.com/stocks/company_info/stock_news.php?sc_id=PSL&scat=&pageno=1&next=0&durationType=Y&Year=2020&duration=1&news_type=
year:  2019
page_no:  1
next:  0
url:  https://www.moneycontrol.com/stocks/company_info/st